# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [25]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from IPython.display import update_display


In [8]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [41]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'

In [42]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 74701a8c35f6: 100% ▕██████████████████▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
# set up environment
gpt = OpenAI(api_key=api_key)
llama = OpenAI(base_url="http://localhost:11434/v1")

In [14]:
system_prompt = """
You are a helpful assistant that can answer questions about the code. and give a detailed explanation. and example.
your response should be in markdown format. and well structured.
"""


In [49]:


def getAnswer(question, model=MODEL_GPT):
    client = llama if model == MODEL_LLAMA else gpt
    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        stream=True,
    )
    display_handle = display(Markdown(""), display_id=True)
    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)


In [38]:
# # Get gpt-4o-mini to answer, with streaming

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
getAnswer(question)



Certainly! Let's break down the provided code snippet:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explanation

1. **Context of Usage**:
   - This code is likely part of a generator function in Python. The `yield from` expression is used to yield all values from an iterable (in this case, a set comprehension).

2. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
   - **`book.get("author")`**:
     - This retrieves the value associated with the key `"author"` from the `book` dictionary.
     - If the key does not exist, it returns `None`.
   - **Iteration**: 
     - The comprehension iterates over a collection named `books`.
     - Each `book` is expected to be a dictionary representing a book with various attributes, including the author.
   - **Filtering**: 
     - The condition `if book.get("author")` filters out any books where the `author` attribute is `None` or missing.
     - Only books with a valid author (non-empty string or non-None value) will contribute to the resulting set.

3. **Creating a Set**:
   - The entirety of the comprehension creates a set of authors, ensuring that each author appears only once.
   - This is important because using a set automatically removes duplicates.

4. **Yielding**:
   - `yield from` is a way in Python to yield multiple values from an iterable within a generator function.
   - It allows the generator to yield all elements of the set produced by the comprehension, one by one.

### Purpose
The purpose of this code is to extract unique authors from a list of books, filtering out any entries that don't have a specified author, and then yielding each unique author one at a time.

### Example

Let's assume we have the following list of books represented as dictionaries:

```python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": None},
    {"title": "Book Four", "author": "Author A"},
    {"title": "Book Five", "author": "Author C"},
    {"title": "Book Six"},
]
```

Here's how the code would work step by step:

1. **Iterate through the list**:
   - For each book, check if the author exists.
2. **Filtered set**:
   - It would create a set from authors:
     - "Author A" (from Book One)
     - "Author B" (from Book Two)
     - "Author C" (from Book Five)
   - The entry for "Author A" from Book Four would be ignored because it is a duplicate in the set.
   - Books without authors or with a `None` value are skipped.
3. **Yielding authors**:
   - The generator would yield:
     - "Author A"
     - "Author B"
     - "Author C"

### Conclusion

This code is an efficient way to work with collections of data (e.g., books), ensuring that you filter and extract unique elements (authors) while maintaining a clean and concise syntax using Python's set comprehensions and generator features. By using `yield from`, it allows seamless integration within larger data processing flows.

In [50]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
getAnswer(question, MODEL_LLAMA)

**Explanation of Code**

The provided code is written in Python and utilizes a feature called "generator expressions" along with the `yield` keyword. Here's a breakdown:

```python
yield from {book.get("author")
    for book in books
    if book.get("author")}

```

* `yield from`: This is a shortcut to express a generator expression inside another generator. It essentially pauses execution of that block and returns the generator object immediately after it yields, similar to how the built-in functions like `filter()` or `map()` work.
  
* `{book.get("author") for book in books if book.get("author")}`: This is an iterator function that takes two arguments - a generator (the iterable provided by `for book in books`) and another condition. The `if` condition filters the books where the author exists.

**Functionality Explanation**

1. It extracts all authors from the list of books where each book has 'author' key in its dictionary.
2. This is done inside an iterator function `(book.get("author") for book in books)`.
3. A generator expression `{yield from {book.get("author") if book.get("author"}}` iterates over this filtered list of books and yields (returns) each author.

However, in Python, the entire `if-else` logic is redundant as shown:

```python
def get_books_with_author():
    """Get books with authors"""
    books = [
            {"title": "Book 1", "author": 'Author A'},
            {"title": "Book 2", "author": 'Author B'},
            {"title": "Book 3"}, 
            # Author D exists
            {'title': 'Book 4', 'author': 'Author C'}
    ]

    yield from [
        book.get("author") for book in books if book.get("author")
    ]
```

So the code actually simplifies to:

```python
def get_books_with_author():
    """Get books with authors"""
    books = [{}  # The dictionary with author as key and value in this case. It will be empty.
            {"title": "Book 1", "author": 'Author A'},
            {"title": "Book 2", "author": 'Author B'},
            {"title": "Book 3"},
            {'title': 'Book 4', 'author': 'Author C'}
    ]
    
    return [book.get("author") for book in books if hasattr(book, "get("author")]]

# Run this function
books_dict = get_books_with_author()
```

In summary, the provided code is equivalent to a more concise and readable way of solving the problem:

```python
def filter_and_get_authors(books):
    yield from [
        book.get("author") 
        for book in books 
        if hasattr(book, "get("author")
    ]
```

In [28]:
# Get Llama 3.2 to answer